In [ ]:
import time
import datetime
start = time.time()

In [ ]:
import loompy
import scanpy as sc
import pandas as pd
import seaborn as sns

In [ ]:
sc.settings.verbosity = 3  
sc.logging.print_header() 
sc.settings.set_figure_params(dpi=80)

In [ ]:
WTAmp = sc.read_loom('../looms-CR5-r-102/WTA.loom')
WTIsth = sc.read_loom('../looms-CR5-r-102/WTI.loom')
cKOAmp = sc.read_loom('../looms-CR5-r-102/cKOA.loom')
cKOIsth = sc.read_loom('../looms-CR5-r-102/cKOI.loom')

In [ ]:
WTAmp.var_names_make_unique()
WTIsth.var_names_make_unique()
cKOAmp.var_names_make_unique()
cKOIsth.var_names_make_unique()

In [ ]:
WTAmpIsth = WTAmp.concatenate(WTIsth, batch_key='Region', batch_categories=['Ampulla','Isthmus'])
cKOAmpIsth = cKOAmp.concatenate(cKOIsth, batch_key='Region', batch_categories=['Ampulla','Isthmus'])

In [ ]:
WTcKOAmpIsth = WTAmpIsth.concatenate(cKOAmpIsth, batch_key='Genotype', batch_categories=['WT','cKO'])

In [ ]:
WTcKOAmpIsth

In [ ]:
WTcKOAmpIsth.write('./WTcKOAmpIsth.h5ad')

In [ ]:
WTAmp.write('./WTInfAmp.h5ad')

In [ ]:
WTIsth.write('./WTIsthUTJ.h5ad')

In [ ]:
cKOAmp.write('./cKOInfAmp.h5ad')

In [ ]:
cKOIsth.write('./cKOIsthUTJ.h5ad')

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import bbknn
from pandas.core.index import RangeIndex
from matplotlib import rcParams
from matplotlib.pyplot import rc_context

In [ ]:
sc.settings.verbosity = 3   
sc.logging.print_header() 
sc.settings.set_figure_params(dpi=80, facecolor='white') 

In [ ]:
!pwd

In [ ]:
WTcKOAmpIsth = sc.read( './WTcKOAmpIsth.h5ad') 

In [ ]:
WTcKOAmpIsth.var_names_make_unique()

In [ ]:
WTcKOAmpIsth

In [ ]:
sc.pl.highest_expr_genes(WTcKOAmpIsth, n_top=20)

In [ ]:
sc.pp.filter_cells(WTcKOAmpIsth, min_genes=200) 
sc.pp.filter_genes(WTcKOAmpIsth, min_cells=3)

In [ ]:

mito_genes = WTcKOAmpIsth.var_names.str.startswith('mt') 
WTcKOAmpIsth.obs['pct_counts_mt'] = np.sum(
    WTcKOAmpIsth[:, mito_genes].X, axis=1).A1 / np.sum(WTcKOAmpIsth.X, axis=1).A1 
WTcKOAmpIsth.obs['total_counts'] = WTcKOAmpIsth.X.sum(axis=1).A1 

In [ ]:
WTcKOAmpIsth

In [ ]:

sc.pl.violin(WTcKOAmpIsth, ['n_genes'], jitter=0.4)
sc.pl.violin(WTcKOAmpIsth, ['total_counts'], jitter=0.4)
sc.pl.violin(WTcKOAmpIsth, ['pct_counts_mt'], jitter=0.4)

In [ ]:
WTcKOAmpIsth

In [ ]:
sc.pl.scatter(WTcKOAmpIsth, x='total_counts', y='pct_counts_mt')
# sc.pl.scatter(WTcKOAmpIsth, x='total_counts', y='n_genes_by_counts')
sc.pl.scatter(WTcKOAmpIsth, x='total_counts', y='n_genes')

In [ ]:
WTcKOAmpIsth

In [ ]:
WTcKOAmpIsth = WTcKOAmpIsth[WTcKOAmpIsth.obs.n_genes < 4000, :]
WTcKOAmpIsth = WTcKOAmpIsth[WTcKOAmpIsth.obs.pct_counts_mt < 0.1, :]

In [ ]:
sc.pp.normalize_total(WTcKOAmpIsth, target_sum=None)

In [ ]:
sc.pp.log1p(WTcKOAmpIsth)

In [ ]:
sc.pp.regress_out(WTcKOAmpIsth, ['total_counts', 'pct_counts_mt'])

In [ ]:
sc.pp.highly_variable_genes(WTcKOAmpIsth, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(WTcKOAmpIsth)

In [ ]:
WTcKOAmpIsth.raw = WTcKOAmpIsth

In [ ]:
WTcKOAmpIsth = WTcKOAmpIsth[:, WTcKOAmpIsth.var.highly_variable]

In [ ]:
sc.pp.scale(WTcKOAmpIsth, max_value=10) 

In [ ]:
sc.tl.pca(WTcKOAmpIsth, svd_solver='arpack')

In [ ]:
sc.pl.pca(WTcKOAmpIsth, color='Genotype')

In [ ]:
sc.pl.pca_variance_ratio(WTcKOAmpIsth, log=True)

In [ ]:
sc.pl.pca(WTcKOAmpIsth, wspace=0.35,
          color=['Region', 'Genotype', 'Epcam', 'Ovgp1', 'Foxj1', 'Myh11', 'Cd52', 'Pecam1', 
                  
                  'Pdgfra', 'Twist2', 'Igf1', 'Dcn', 'Serpina1e', 'Myb', 'Pax8', 'Pax2', 'Cxcl17', 'Id2', 'Pdxk', 'Ccdc153'])

In [ ]:
sc.pl.pca_variance_ratio(WTcKOAmpIsth, n_pcs=37, log=True)

In [ ]:
WTcKOAmpIsth

In [ ]:

sc.pp.neighbors(WTcKOAmpIsth, n_neighbors=10, n_pcs=37)

In [ ]:

sc.tl.leiden(WTcKOAmpIsth)
sc.tl.paga(WTcKOAmpIsth)
sc.pl.paga(WTcKOAmpIsth, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(WTcKOAmpIsth, init_pos='paga')

In [ ]:

sc.tl.umap(WTcKOAmpIsth)

In [ ]:

sc.tl.leiden(WTcKOAmpIsth, resolution=0.3) 

In [ ]:
sc.pl.umap(WTcKOAmpIsth,color=['leiden','Genotype', 'Region'],legend_loc='on data')

In [ ]:

sc.pl.umap(WTcKOAmpIsth,
           color=['leiden', 'Region', 'Genotype', 'Epcam', 'Ovgp1', 'Foxj1', 'Myh11', 'Cd52', 'Ptprc', 'Pecam1', 
                  
                  'Pdgfra', 'Twist2', 'Igf1', 'Dcn', 'Serpina1e', 'Myb', 'Pax8', 'Pax2', 'Cxcl17', 'Id2', 'Pdxk', 'Ccdc153', 'Il22'])

In [ ]:
WTcKOAmpIsth.obs.groupby(["leiden","Genotype","Region"]).apply(len)

In [ ]:

marker_genes = ['Epcam', 'Krt8','Krt19','Vim','Pdgfra','Twist2', 'Sprr2f',
                'Acta2', 'Myh11','Foxj1', 'Ccdc153', 'Ovgp1','Wt1','Serpina1e',
                'S100g','Pecam1', 'Ptprc', 'Pgr', 'Esr1', 'Crabp2', 'Cd44', 'Pdxk', 
                 ]

In [ ]:
sc.pl.umap(WTcKOAmpIsth, color=marker_genes)

In [ ]:
WTcKOAmpIsth.obs.groupby(["leiden","Genotype","Region"]).apply(len)

In [ ]:
new_cluster_names = [
     '0 Secretory_IsthUTJ',
     '1 Secretory_InfAmpIsthUTJ',
     '2 Ciliated_InfAmp',
     '3 Secretory_InfAmp',
     '4 Fibroblast_Pdgfra-',
     '5 Ciliated_IsthUTJ',
     '6 Fibroblast_Pdgfra+',
     '7 Muscle',
     '8 Immune1',
     '9 Secretory_IsthUTJ_subset',
     '10 Endothelial',
    '11 Immune2',
    '12 Unidentified1',
      '13 Unidentified2'
]
WTcKOAmpIsth.rename_categories('leiden', new_cluster_names)

In [ ]:
plt.figure(figsize=(8, 2))
for i in range(28):
    plt.scatter(i, 1, c=sc.pl.palettes.zeileis_28[i], s=200)
plt.show()

In [ ]:
zeileis_colors = np.array(sc.pl.palettes.zeileis_28)
new_colors = np.array(WTcKOAmpIsth.uns['leiden_colors'])

In [ ]:
new_colors[[0]] = zeileis_colors[[7]]  
new_colors[[1]] = zeileis_colors[[0]]  
new_colors[[2]] = zeileis_colors[[5]]
new_colors[[3]] = zeileis_colors[[6]]  
new_colors[[4]] = zeileis_colors[[16]] 
new_colors[[5]] = zeileis_colors[[4]] 
new_colors[[6]] = zeileis_colors[[17]] 
new_colors[[7]] = zeileis_colors[[24]] 
new_colors[[8]] = zeileis_colors[[12]] 
new_colors[[9]] = zeileis_colors[[8]] 
new_colors[[10]] = zeileis_colors[[26]] 
new_colors[[11]] = zeileis_colors[[13]] 
new_colors[[12]] = zeileis_colors[[20]] 
new_colors[[13]] = zeileis_colors[[19]] 

In [ ]:
WTcKOAmpIsth.uns['leiden_colors'] = new_colors

In [ ]:
sc.pl.umap(WTcKOAmpIsth, color='leiden', legend_loc='right margin', legend_fontsize='8.0',
           frameon=False, save='Wnt7aPgr_0.5dpc_Leiden_AllPops060424.pdf')

In [ ]:
new_cluster_names = [
    'WT',
    'cKO'
]
WTcKOAmpIsth.rename_categories('Genotype', new_cluster_names)

In [ ]:
plt.figure(figsize=(8, 2))
for i in range(28):
    plt.scatter(i, 1, c=sc.pl.palettes.zeileis_28[i], s=200)
plt.show()

In [ ]:
new_cluster_names = [
    'WT',
    'cKO'
]
WTcKOAmpIsth.rename_categories('Genotype', new_cluster_names)

In [ ]:
zeileis_colors = np.array(sc.pl.palettes.zeileis_28)
new_colors = np.array(WTcKOAmpIsth.uns['Genotype_colors'])

In [ ]:
new_colors[[0]] = zeileis_colors[[0]]  #0 WT
new_colors[[1]] = zeileis_colors[[17]]  #1 cKO

In [ ]:
WTcKOAmpIsth.uns['Genotype_colors'] = new_colors

In [ ]:
sc.pl.umap(WTcKOAmpIsth, color='Genotype', legend_loc='right margin', legend_fontsize='8.0',
           frameon=False, save='Wnt7aPgr_0.5dpc_Genotype_AllPops060424.pdf')

In [ ]:
new_cluster_names = [
    'InfAmp',
    'IsthUTJ'
]
WTcKOAmpIsth.rename_categories('Region', new_cluster_names)

In [ ]:
plt.figure(figsize=(8, 2))
for i in range(28):
    plt.scatter(i, 1, c=sc.pl.palettes.zeileis_28[i], s=200)
plt.show()

In [ ]:
zeileis_colors = np.array(sc.pl.palettes.zeileis_28)
new_colors = np.array(WTcKOAmpIsth.uns['Region_colors'])

In [ ]:
new_colors[[0]] = zeileis_colors[[9]]  #0 InfAmp
new_colors[[1]] = zeileis_colors[1]  #1 IsthUTJ

In [ ]:
WTcKOAmpIsth.uns['Region_colors'] = new_colors

In [ ]:
sc.pl.umap(WTcKOAmpIsth, color='Region', legend_loc='right margin', legend_fontsize='8.0',
           frameon=False, save='Wnt7aPgr_0.5dpc_Region_AllPops060424.pdf')

In [ ]:
ax = sc.pl.dotplot(WTcKOAmpIsth, 
                   marker_genes, groupby='leiden', dendrogram=True, color_map='Blues',
                  save='_Wnt7aPgr 0.5dpc_AllPops060424')

In [ ]:
ax = sc.pl.dotplot(WTcKOAmpIsth, 
                   marker_genes, groupby='Genotype', dendrogram=True, color_map='Blues',
                  save='_Wnt7aPgr 0.5dpc Genoptype_AllPops060424')

In [ ]:
sc.tl.rank_genes_groups(WTcKOAmpIsth, 'Genotype', method='wilcoxon',n_genes=1000)
sc.pl.rank_genes_groups(WTcKOAmpIsth, n_genes=1000, sharey=False)
df = pd.DataFrame(WTcKOAmpIsth.uns['rank_genes_groups']['names']).head(1000)
df.to_csv('./WTcKOAmpIsth_Genotype_AllPops060424_top1000.csv')

In [ ]:
#subsetting
WTcKOAmpIsth_IA = WTcKOAmpIsth[WTcKOAmpIsth.obs['Region'].isin(['0 InfAmp'])]
WTcKOAmpIsth_IU = WTcKOAmpIsth[WTcKOAmpIsth.obs['Region'].isin(['1 IsthUTJ'])]
WTcKOAmpIsth_Immune = WTcKOAmpIsth[WTcKOAmpIsth.obs['leiden'].isin(['8 Immune1','11 Immune2'])]
WTcKOAmpIsth_Muscle = WTcKOAmpIsth[WTcKOAmpIsth.obs['leiden'].isin(['7 Muscle'])]
WTcKOAmpIsth_Secretory = WTcKOAmpIsth[WTcKOAmpIsth.obs['leiden'].isin(['0 Secretory_IsthUTJ','1 Secretory_InfAmpIsthUTJ',
                                                                   '3 Secretory_InfAmp','9 Secretory_IsthUTJ_subset'])]
WTcKOAmpIsth_Ciliated = WTcKOAmpIsth[WTcKOAmpIsth.obs['leiden'].isin(['2 Ciliated_InfAmp','5 Ciliated_IsthUTJ'])]
WTcKOAmpIsth_Fibroblast = WTcKOAmpIsth[WTcKOAmpIsth.obs['leiden'].isin(['4 Fibroblast_Pdgfra-','6 Fibroblast_Pdgfra+'])]
WTcKOAmpIsth_Epithelial = WTcKOAmpIsth[WTcKOAmpIsth.obs['leiden'].isin([
    '0 Secretory_IsthUTJ','1 Secretory_InfAmpIsthUTJ','3 Secretory_InfAmp','9 Secretory_IsthUTJ_subset',
'2 Ciliated_InfAmp','5 Ciliated_IsthUTJ'])]


In [ ]:
WTcKOAmpIsth_IA

In [ ]:
WTcKOAmpIsth_IU

In [ ]:
WTcKOAmpIsth_Immune

In [ ]:
WTcKOAmpIsth_Muscle

In [ ]:
WTcKOAmpIsth_Secretory

In [ ]:
WTcKOAmpIsth_Ciliated

In [ ]:
WTcKOAmpIsth_Fibroblast

In [ ]:
WTcKOAmpIsth_Epithelial

In [ ]:

sc.tl.pca(WTcKOAmpIsth_Immune, svd_solver='arpack')
sc.tl.pca(WTcKOAmpIsth_Muscle, svd_solver='arpack')
sc.tl.pca(WTcKOAmpIsth_Secretory, svd_solver='arpack')
sc.tl.pca(WTcKOAmpIsth_Ciliated, svd_solver='arpack')
sc.tl.pca(WTcKOAmpIsth_Fibroblast, svd_solver='arpack')
sc.tl.pca(WTcKOAmpIsth_Epithelial, svd_solver='arpack')


In [ ]:
sc.pl.pca_variance_ratio(WTcKOAmpIsth_Immune, n_pcs=37, log=True)
sc.pl.pca_variance_ratio(WTcKOAmpIsth_Muscle, n_pcs=37, log=True)
sc.pl.pca_variance_ratio(WTcKOAmpIsth_Secretory, n_pcs=37, log=True)
sc.pl.pca_variance_ratio(WTcKOAmpIsth_Ciliated, n_pcs=37, log=True)
sc.pl.pca_variance_ratio(WTcKOAmpIsth_Fibroblast, n_pcs=37, log=True)
sc.pl.pca_variance_ratio(WTcKOAmpIsth_Epithelial, n_pcs=37, log=True)


In [ ]:
sc.pp.neighbors(WTcKOAmpIsth_Immune, n_neighbors=10, n_pcs=37)
sc.pp.neighbors(WTcKOAmpIsth_Muscle, n_neighbors=10, n_pcs=37)
sc.pp.neighbors(WTcKOAmpIsth_Secretory, n_neighbors=10, n_pcs=37)
sc.pp.neighbors(WTcKOAmpIsth_Ciliated, n_neighbors=10, n_pcs=37)
sc.pp.neighbors(WTcKOAmpIsth_Fibroblast, n_neighbors=10, n_pcs=37)
sc.pp.neighbors(WTcKOAmpIsth_Epithelial, n_neighbors=10, n_pcs=37)


In [ ]:
sc.tl.umap(WTcKOAmpIsth_Immune)
sc.tl.umap(WTcKOAmpIsth_Muscle)
sc.tl.umap(WTcKOAmpIsth_Secretory)
sc.tl.umap(WTcKOAmpIsth_Ciliated)
sc.tl.umap(WTcKOAmpIsth_Fibroblast)
sc.tl.umap(WTcKOAmpIsth_Epithelial)

In [ ]:
gene_dict = ['Epcam', 'Ovgp1', 'Foxj1', 'Myh11', 'Ptprc', 'Pecam1', 'Myh11',
                  
                  'Pdgfra', 'Twist2', 'Igf1', 'Dcn', 'Serpina1e', 'Myb', 'Pax8', 'Pax2', 'Cxcl17', 'Id2', 
             'Pdxk', 'Ccdc153', 'Il22', 'Il22ra1', 'Il22ra2', 'Stat3']

In [ ]:
sc.pl.umap(WTcKOAmpIsth_Immune, color=gene_dict)
sc.pl.umap(WTcKOAmpIsth_Muscle, color=gene_dict)
sc.pl.umap(WTcKOAmpIsth_Secretory, color=gene_dict)
sc.pl.umap(WTcKOAmpIsth_Ciliated, color=gene_dict)
sc.pl.umap(WTcKOAmpIsth_Fibroblast, color=gene_dict)
sc.pl.umap(WTcKOAmpIsth_Epithelial, color=gene_dict)

In [ ]:
sc.pl.umap(WTcKOAmpIsth_Immune, color='leiden', legend_loc='right margin', legend_fontsize='8.0',
           frameon=False, save='Wnt7aPgr_0.5dpc_ImmuneOnly_Leiden060724.pdf')

In [ ]:
#WT and cKO Genes of interest - immune cells
sc.pl.dotplot(WTcKOAmpIsth_Immune, ['Il1b', 'Il22', 'Il22ra2', 'Il10', 'Il15', 'Il17a', 'Il18', 'Ifng', 'Tlr2', 'Tlr3', 
                                                   'Tlr4', 'Tlr5', 'Tlr6', 'Tlr7', 'Tlr8', 'Tlr9', 'Ccl24'],
groupby='Genotype', color_map='Blues', 
             save='_WTcKOAmpIsth_Immune_Genes of interest_immune genes_WTvsCKO_20240606.pdf')

In [ ]:
#WT and cKO Genes of interest - immune cells
sc.pl.dotplot(WTcKOAmpIsth_Immune, ['Il22', 'Il22ra2', 'Il10', 'Il15', 'Il17a', 'Il18', 'Ifng', 'Tlr2', 'Tlr3', 
                                                   'Tlr4', 'Tlr5', 'Tlr6', 'Tlr7', 'Tlr8', 'Tlr9', 'Ccl24'],
groupby='Genotype', color_map='Blues', 
             save='_WTcKOAmpIsth_Immune_Genes of interest_immune genes2_WTvsCKO_20240606.pdf')

In [ ]:
#WT and cKO Genes of interest - immune cells
sc.pl.dotplot(WTcKOAmpIsth_Epithelial, ['Il1b', 'Il22', 'Il22ra2', 'Il10', 'Il15', 'Il17a', 'Il18', 'Ifng', 'Tlr2', 'Tlr3', 
                                                   'Tlr4', 'Tlr5', 'Tlr6', 'Tlr7', 'Tlr8', 'Tlr9', 'Ccl24'],
groupby='Genotype', color_map='Blues', 
             save='_WTcKOAmpIsth_Epithelial_Genes of interest_immune genes2_WTvsCKO_20240606.pdf')

In [ ]:
#WT and cKO Genes of interest - immune cells
sc.pl.dotplot(WTcKOAmpIsth_Secretory, ['Il1b', 'Il22', 'Il22ra2', 'Il10', 'Il15', 'Il17a', 'Il18', 'Ifng', 'Tlr2', 'Tlr3', 
                                                   'Tlr4', 'Tlr5', 'Tlr6', 'Tlr7', 'Tlr8', 'Tlr9', 'Ccl24'],
groupby='Genotype', color_map='Blues', 
             save='_WTcKOAmpIsth_Secretory_Genes of interest_immune genes2_WTvsCKO_20240606.pdf')

In [ ]:
#WT and cKO Genes of interest - immune cells
sc.pl.dotplot(WTcKOAmpIsth_Fibroblast, ['Il1b', 'Il22', 'Il22ra2', 'Il10', 'Il15', 'Il17a', 'Il18', 'Ifng', 'Tlr2', 'Tlr3', 
                                                   'Tlr4', 'Tlr5', 'Tlr6', 'Tlr7', 'Tlr8', 'Tlr9', 'Ccl24'],
groupby='Genotype', color_map='Blues', 
             save='_WTcKOAmpIsth_Fibroblast_Genes of interest_immune genes2_WTvsCKO_20240606.pdf')

In [ ]:
#WT and cKO Genes of interest - immune cells
sc.pl.dotplot(WTcKOAmpIsth_Ciliated, ['Il1b', 'Il22', 'Il22ra2', 'Il10', 'Il15', 'Il17a', 'Il18', 'Ifng', 'Tlr2', 'Tlr3', 
                                                   'Tlr4', 'Tlr5', 'Tlr6', 'Tlr7', 'Tlr8', 'Tlr9', 'Ccl24'],
groupby='Genotype', color_map='Blues', 
             save='_WTcKOAmpIsth_Ciliated_Genes of interest_immune genes2_WTvsCKO_20240606.pdf')

In [ ]:
markers = ['Il1b', 'Il22', 'Il10', 'Il18', 'Ifng', 'Tlr2', 'Tlr4']
sc.pl.stacked_violin(WTcKOAmpIsth_Epithelial, markers, groupby='leiden', dendrogram=False, color_map='Blues')

In [ ]:
markers = ['Il1b', 'Il22', 'Il10', 'Il18', 'Ifng', 'Tlr2', 'Tlr4']
sc.pl.stacked_violin(WTcKOAmpIsth_Epithelial, markers, groupby='Genotype', dendrogram=True, color_map='Blues')

In [ ]:
sc.pl.dotplot(WTcKOAmpIsth_Immune, ['Il1b', 'Il10', 'Il15', 'Il17a', 'Il18', 'Il22', 'Ifng', 'Tlr2', 'Tlr4'], groupby='Genotype', color_map='Blues', 
             save='_WTcKOAmpIsth_Immune_IL1b,10,15,17a,18,22,Ifng,Tlr2,Tlr4_WTvsCKO_20240606.pdf')
                                    
sc.pl.dotplot(WTcKOAmpIsth_Epithelial, ['Il1b', 'Il10', 'Il15', 'Il17a', 'Il18', 'Il22', 'Ifng', 'Tlr2', 'Tlr4'], groupby='Genotype', color_map='Blues', 
             save='_WTcKOAmpIsth_Epithelial_IL1b,10,15,17a,18,22,Ifng,Tlr2,Tlr4_WTvsCKO_20240606.pdf')

sc.pl.dotplot(WTcKOAmpIsth_Secretory, ['Il1b', 'Il10', 'Il15', 'Il17a', 'Il18', 'Il22', 'Ifng', 'Tlr2', 'Tlr4'], groupby='Genotype', color_map='Blues', 
             save='_WTcKOAmpIsth_Secretory_IL1b,10,15,17a,18,22,Ifng,Tlr2,Tlr4_WTvsCKO_20240606.pdf')
                                    
sc.pl.dotplot(WTcKOAmpIsth_Ciliated, ['Il1b', 'Il10', 'Il15', 'Il17a', 'Il18', 'Il22', 'Ifng', 'Tlr2', 'Tlr4'], groupby='Genotype', color_map='Blues', 
             save='_WTcKOAmpIsth_Ciliated_IL1b,10,15,17a,18,22,Ifng,Tlr2,Tlr4_WTvsCKO_20240606.pdf')

sc.pl.dotplot(WTcKOAmpIsth_Fibroblast, ['Il1b', 'Il10', 'Il15', 'Il17a', 'Il18', 'Il22', 'Ifng', 'Tlr2', 'Tlr4'], groupby='Genotype', color_map='Blues', 
             save='_WTcKOAmpIsth_Fibroblast_IL1b,10,15,17a,18,22,Ifng,Tlr2,Tlr4_WTvsCKO_20240606.pdf')

In [ ]:
sc.pl.dotplot(WTcKOAmpIsth_Immune, ['Il10ra', 'Il10rb','Stat1','Stat3','Map2k1','Mapk8','Mapk14','Jak1','Tyk2'], groupby='Genotype', color_map='Blues', 
             save='_WTcKOAmpIsth_Immune_IL22Signal_WTvsCKO_20240606.pdf')
                                    
sc.pl.dotplot(WTcKOAmpIsth_Epithelial, ['Il10ra', 'Il10rb','Stat1','Stat3','Map2k1','Mapk8','Mapk14','Jak1','Tyk2'], groupby='Genotype', color_map='Blues', 
             save='_WTcKOAmpIsth_Epithelial_IL22Signal_WTvsCKO_20240606.pdf')

sc.pl.dotplot(WTcKOAmpIsth_Secretory, ['Il10ra', 'Il10rb','Stat1','Stat3','Map2k1','Mapk8','Mapk14','Jak1','Tyk2'], groupby='Genotype', color_map='Blues', 
             save='_WTcKOAmpIsth_Secretory_IL22Signal_WTvsCKO_20240606.pdf')
                                    
sc.pl.dotplot(WTcKOAmpIsth_Ciliated, ['Il10ra', 'Il10rb','Stat1','Stat3','Map2k1','Mapk8','Mapk14','Jak1','Tyk2'], groupby='Genotype', color_map='Blues', 
             save='_WTcKOAmpIsth_Ciliated_IL22Signal_20240606.pdf')

sc.pl.dotplot(WTcKOAmpIsth_Fibroblast, ['Il10ra', 'Il10rb','Stat1','Stat3','Map2k1','Mapk8','Mapk14','Jak1','Tyk2'], groupby='Genotype', color_map='Blues', 
             save='_WTcKOAmpIsth_Fibroblast_IL22Signal_20240606.pdf')

In [ ]:
sc.tl.rank_genes_groups(WTcKOAmpIsth_Immune, 'Genotype', method='wilcoxon')
sc.tl.rank_genes_groups(WTcKOAmpIsth_Epithelial, 'Genotype', method='wilcoxon')
sc.tl.rank_genes_groups(WTcKOAmpIsth_Secretory, 'Genotype', method='wilcoxon')
sc.tl.rank_genes_groups(WTcKOAmpIsth_Ciliated, 'Genotype', method='wilcoxon')
sc.tl.rank_genes_groups(WTcKOAmpIsth_Fibroblast, 'Genotype', method='wilcoxon')


In [ ]:
sc.tl.dendrogram(WTcKOAmpIsth, 'leiden')

In [ ]:
ax = sc.pl.dendrogram(WTcKOAmpIsth, 'leiden')

In [ ]:
sc.tl.rank_genes_groups(WTcKOAmpIsth_Immune, 'Genotype', method='wilcoxon',n_genes=1000)
sc.pl.rank_genes_groups(WTcKOAmpIsth_Immune, n_genes=1000, sharey=False)
df_InfAmp = pd.DataFrame(WTcKOAmpIsth_Immune.uns['rank_genes_groups']['names']).head(1000)
df_InfAmp.to_csv('./WTcKOAmpIsth_Immune_Genotype_top1000_060624.csv')

In [ ]:
sc.tl.rank_genes_groups(WTcKOAmpIsth_Epithelial, 'Genotype', method='wilcoxon',n_genes=1000)
sc.pl.rank_genes_groups(WTcKOAmpIsth_Epithelial, n_genes=1000, sharey=False)
df_InfAmp = pd.DataFrame(WTcKOAmpIsth_Epithelial.uns['rank_genes_groups']['names']).head(1000)
df_InfAmp.to_csv('./WTcKOAmpIsth_Epithelial_Genotype_top1000_060624.csv')

In [ ]:
sc.tl.rank_genes_groups(WTcKOAmpIsth_Fibroblast, 'Genotype', method='wilcoxon',n_genes=1000)
sc.pl.rank_genes_groups(WTcKOAmpIsth_Fibroblast, n_genes=1000, sharey=False)
df_InfAmp = pd.DataFrame(WTcKOAmpIsth_Fibroblast.uns['rank_genes_groups']['names']).head(1000)
df_InfAmp.to_csv('./WTcKOAmpIsth_Fibroblast_Genotype_top1000_060624.csv')

In [ ]:
sc.tl.rank_genes_groups(WTcKOAmpIsth_Secretory, 'Genotype', method='wilcoxon',n_genes=1000)
sc.pl.rank_genes_groups(WTcKOAmpIsth_Secretory, n_genes=1000, sharey=False)
df_InfAmp = pd.DataFrame(WTcKOAmpIsth_Secretory.uns['rank_genes_groups']['names']).head(1000)
df_InfAmp.to_csv('./WTcKOAmpIsth_Secretory_Genotype_top1000_060624.csv')

In [ ]:
sc.tl.rank_genes_groups(WTcKOAmpIsth_Ciliated, 'Genotype', method='wilcoxon',n_genes=1000)
sc.pl.rank_genes_groups(WTcKOAmpIsth_Ciliated, n_genes=1000, sharey=False)
df_InfAmp = pd.DataFrame(WTcKOAmpIsth_Ciliated.uns['rank_genes_groups']['names']).head(1000)
df_InfAmp.to_csv('./WTcKOAmpIsth_Ciliated_Genotype_top1000_060624.csv')

In [ ]:
sc.tl.rank_genes_groups(WTcKOAmpIsth_Muscle, 'Genotype', method='wilcoxon',n_genes=1000)
sc.pl.rank_genes_groups(WTcKOAmpIsth_Muscle, n_genes=1000, sharey=False)
df_InfAmp = pd.DataFrame(WTcKOAmpIsth_Muscle.uns['rank_genes_groups']['names']).head(1000)
df_InfAmp.to_csv('./WTcKOAmpIsth_Muscle_Genotype_top1000_060624.csv')